In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("sales")


In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
    SELECT YEAR(sales.date) AS year, ROUND(AVG(price), 2) AS average_price
    FROM sales
    WHERE bedrooms = 4
    GROUP BY YEAR(sales.date)
    ORDER BY year
"""

result = spark.sql(query)
result.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
    SELECT YEAR(sales.date) AS year, ROUND(AVG(price), 2) AS average_price
    FROM sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY YEAR(sales.date)
    ORDER BY year
"""

result = spark.sql(query)
result.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|    287287.82|
|2020|    294204.16|
|2021|    294211.46|
|2022|    292725.69|
+----+-------------+



In [7]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query = """
    SELECT YEAR(sales.date) AS year, ROUND(AVG(price), 2) AS average_price
    FROM sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_lot >= 2000
    GROUP BY YEAR(sales.date)
    ORDER BY year
"""

result = spark.sql(query)
result.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|    282752.72|
|2020|    293176.41|
|2021|    290319.06|
|2022|    287162.18|
+----+-------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
    SELECT ROUND(AVG(price), 2) AS average_price
    FROM sales
    WHERE price >= 350000
"""

result = spark.sql(query)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|average_price|
+-------------+
|    473796.26|
+-------------+

--- 0.12771201133728027 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("sales")

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
query = """
    SELECT ROUND(AVG(price), 2) AS average_price
    FROM sales
    WHERE price >= 350000
"""

start_time = time.time()

result = spark.sql(query)
result.show()


print("--- %s seconds ---" % (time.time() - start_time))


+-------------+
|average_price|
+-------------+
|    473796.26|
+-------------+

--- 0.3670179843902588 seconds ---


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").parquet("data.parquet")

Py4JJavaError: An error occurred while calling o50.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:261)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)

In [ ]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet("data.parquet")

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_data")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()
query = """
    SELECT ROUND(AVG(price), 2) AS average_price
    FROM parquet_data
    WHERE price >= 350000
"""

result = spark.sql(query)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("sales")


In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("sales")
print(is_cached)